In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# SVM Library
from sklearn import svm

# Libraries to scale the data
from sklearn.preprocessing import StandardScaler

# Cross Validation
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

from sklearn.decomposition import PCA

# Bayesian Optimization libraries
from functools import partial
from skopt import gp_minimize
from skopt import space
from sklearn import model_selection

# Performance Metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

# Ignores runtime warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

%matplotlib inline

### Analysis on K-means Imputed and Outlier Presence dataset

In [ ]:
df1 = pd.read_csv("../processed-datasets/data_outliers_handled_kmeans_imputed.csv")

In [ ]:
df1.head()

In [ ]:
df1.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df1.sample(5)

In [ ]:
X = df1.drop('Feature', axis=1).values
y = df1.Feature.values.astype(int)

### Splitting the dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

### Scaling the data using Standard Scaler

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled.shape

### Base Model without outliers and K-means Imputed

In [ ]:
svm_base = svm.SVC()

In [ ]:
svm_base.fit(X_train_scaled, y_train)

In [ ]:
y_pred = svm_base.predict(X_test_scaled)

In [ ]:
print("Accuracy of base SVM model on test data: ", round(accuracy_score(y_test, y_pred), 2))

In [ ]:
print("Classification Report of SVM base model over test data: \n\n", classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(cm, display_labels=['Barren', 'Fertile']).plot()

### Hyper-Parameter Optimization - Grid Search and Stratified CV

In [ ]:
Stratified_kf = StratifiedKFold(n_splits=10, shuffle=True)

In [ ]:
param_grid = {
    'C': np.arange(0, 6, 1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': np.arange(0, 3, 1),
    'gamma': ['scale', 'auto']
}

In [ ]:
clf = GridSearchCV(estimator = svm.SVC(), param_grid = param_grid, scoring = 'accuracy', n_jobs=-1, 
                   cv = Stratified_kf, verbose = 10, return_train_score=True)

In [ ]:
clf.fit(X_train_scaled, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
best_params = clf.best_params_

In [ ]:
best_params

In [ ]:
clf.best_score_

In [ ]:
best_model_svc = svm.SVC(**best_params)

In [ ]:
best_model_svc.fit(X_train_scaled, y_train)

In [ ]:
y_pred_best = best_model_svc.predict(X_test_scaled)

In [ ]:
print("Accuracy of SVC tuned model over test set: ", round(accuracy_score(y_test, y_pred_best), 2))

In [ ]:
print("Classification Report of SVM tuned model over test data: \n\n", classification_report(y_test, y_pred_best))

In [ ]:
cm = confusion_matrix(y_test, y_pred_best)

ConfusionMatrixDisplay(cm, display_labels=['Barren', 'Fertile']).plot()